In [ ]:
# Data for Problem 1 (in millions of dollars)
borrow_data = {
    40: {'amount': 15, 'interest': 0.0075, 'expense_days': 11}, # Days 40 to 50
    30: {'amount': 25, 'interest': 0.009, 'expense_days': 10}, # Days 30 to 39
    20: {'amount': 20, 'interest': 0.008, 'expense_days': 10}, # Days 20 to 29
    10: {'amount': 35, 'interest': 0.01, 'expense_days': 10},  # Days 10 to 19
}
initial_cash = 20
Daily_Expense = 1
Borrow_Dates = [40, 30, 20, 10]

# f_k[C] stores the minimum interest from stage k to the end, given cash C at start of day D_k.
# Policy_k[C] stores the optimal decision at state C.
f = {}
policy = {}

# --- Backward Induction ---

# Stage 4 (Day 40) - Base Case
D4 = 40
B4 = borrow_data[D4]['amount']
i4 = borrow_data[D4]['interest']
E4 = borrow_data[D4]['expense_days']
f[D4] = {}
policy[D4] = {}

# C_min_avoid_borrow: minimum cash needed at end of day 39 to cover all expenses (11 days)
C_min_avoid_borrow = E4
interest_if_borrow = B4 * i4

# Since cash can be up to 50M (Initial 20 + max borrow 35) but we only care up to 21
# to establish the threshold for D3, we consider cash states S up to 21.
for S in range(22):
    if S >= C_min_avoid_borrow:
        # Option: Do Not Borrow (Cost: 0)
        f[D4][S] = 0
        policy[D4][S] = "Do Not Borrow"
    else:
        # Option: Must Borrow (Cost: interest_if_borrow)
        # Note: If S < 0, this path is infeasible. For simplicity, we assume we only reach S >= 0.
        f[D4][S] = interest_if_borrow
        policy[D4][S] = f"Borrow {B4}M"

# Stages 3, 2, 1
for idx in range(1, 4):
    D_current = Borrow_Dates[idx]
    D_next = Borrow_Dates[idx-1]

    B_curr = borrow_data[D_current]['amount']
    i_curr = borrow_data[D_current]['interest']
    E_curr = borrow_data[D_current]['expense_days']

    f[D_current] = {}
    policy[D_current] = {}

    # Iterate over relevant cash states S (start of day D_current)
    # Max cash at D10 decision is 20M (end of D9). Max relevant cash is 21M (C3_min_0)
    max_S = 45 if D_current == 10 else 30 # A safe upper bound for calculation

    for S in range(max_S + 1):
        if S < E_curr:
            # Must Borrow (Option 1 is the only feasible choice to cover the expenses before next decision)
            # This is an approximation as an infeasible state should force borrowing.
            # In the final calculation, this is only used for S=10 at D20 decision, where S-E_curr=0.
            # We rely on the core logic for the optimal path.
            pass

        # 1. Option 0: Do Not Borrow at D_current
        # Cash at end of D_current + E_curr - 1 is S' = S - E_curr
        S_prime_0 = S - E_curr

        cost_0 = float('inf')
        if S_prime_0 >= 0:
            # Total Interest = 0 (current) + f_next(S')
            cost_0 = 0 + f[D_next].get(S_prime_0, float('inf'))

        # 2. Option 1: Borrow B_curr at D_current
        # Cash at end of D_current + E_curr - 1 is S' = S - E_curr + B_curr
        S_prime_1 = S - E_curr + B_curr

        # Total Interest = (B_curr * i_curr) (current) + f_next(S')
        # S' is always non-negative as B_curr >= E_curr.
        cost_1 = (B_curr * i_curr) + f[D_next].get(S_prime_1, float('inf'))

        # Choose the minimum cost
        if cost_0 <= cost_1:
            f[D_current][S] = cost_0
            policy[D_current][S] = "Do Not Borrow"
        else:
            f[D_current][S] = cost_1
            policy[D_current][S] = f"Borrow {B_curr}M"

# --- Determine Final Plan (Forward Pass) ---
current_cash = initial_cash - (Borrow_Dates[-1] - 1) * Daily_Expense # Cash at start of D10 (end of D9): 20 - 9 = 11

final_plan = []
total_interest = 0

for D_current in [10, 20, 30, 40]:
    decision = policy[D_current][current_cash]
    final_plan.append((D_current, decision))

    # Update total interest and cash state based on decision
    B_curr = borrow_data[D_current]['amount']
    i_curr = borrow_data[D_current]['interest']
    E_curr = borrow_data[D_current]['expense_days']

    if decision != "Do Not Borrow":
        total_interest += B_curr * i_curr
        current_cash = current_cash - E_curr + B_curr
    else:
        current_cash = current_cash - E_curr

print("--- Problem 1: Borrowing Plan (Minimize Interest) ---")
print(f"Initial Cash (start of day 1): ${initial_cash}M")
print(f"Initial Cash State (end of day 9): ${initial_cash - 9}M")
print(f"Minimum Total Interest: ${f[10][initial_cash - 9]:.4f}M")
print("Optimal Borrowing Plan:")
for D, decision in final_plan:
    print(f"  Day {D}: {decision}")


In [2]:
# Data for Problem 2: (Cost, Profit) for each project (1, 2, 3) in each region (1 to 5).
# Stored as a dictionary where key is Region index, value is a list of (Cost, Profit) for projects 1, 2, 3.
project_data = {
    5: [(1, 6), (15, 2), (3, 22)],
    4: [(0, 3), (10, 1), (3, 25)],
    3: [(15, 2), (4, 26), (5, 40)],
    2: [(20, 3), (2, 14), (7, 1)],
    1: [(1, 8), (15, 2), (25, 3)],
}
Budget = 10
Regions = 5

# DP_Table[k][S] = (max_profit_k_to_5, best_project_k) for state S at stage k
# Initialize with a dummy stage 6 (k=6) where the profit is 0 for any budget.
DP_Table = {Regions + 1: {s: (0, 0) for s in range(Budget + 1)}}

# --- Backward Induction (k = 5 down to 1) ---
for k in range(Regions, 0, -1):
    DP_Table[k] = {}

    # Iterate over all possible budget states S for the current stage k
    for S in range(Budget + 1):
        max_profit_total = -1 # Represents an infeasible state
        best_project = -1

        # Iterate over all projects p for the current region k
        for p_idx, (c, p) in enumerate(project_data[k]):
            project_num = p_idx + 1

            # Check feasibility: Cost c must be within the current budget S
            if c <= S:
                # Calculate future profit from stage k+1 with remaining budget S-c
                future_profit, _ = DP_Table[k+1].get(S - c, (-1, -1))

                # If the path to the end (k+1 onwards) is feasible
                if future_profit != -1:
                    current_profit_total = p + future_profit

                    if current_profit_total > max_profit_total:
                        max_profit_total = current_profit_total
                        best_project = project_num

        # Store the optimal value and policy for state S at stage k
        DP_Table[k][S] = (max_profit_total, best_project)

# --- Forward Recursion (Extract Optimal Policy) ---
current_budget = Budget
final_policy = {}
total_profit = 0
current_stage_feasible = True

if DP_Table[1][Budget][0] == -1:
    print(f"Maximum Total Profit: Infeasible. Budget of ${Budget}M is too low.")
    current_stage_feasible = False

if current_stage_feasible:
    for k in range(1, Regions + 1):
        (profit, project_num) = DP_Table[k][current_budget]

        final_policy[k] = f"Project {project_num}"

        c_selected = project_data[k][project_num - 1][0]
        p_selected = project_data[k][project_num - 1][1]

        current_budget -= c_selected
        total_profit += p_selected

    print("--- Problem 2: Project Selection (Maximize Total Profit) ---")
    print(f"Initial Budget: ${Budget}M")
    print(f"Maximum Total Profit: ${total_profit}M")
    print(f"Remaining Budget: ${current_budget}M")
    print("Optimal Investment Plan:")
    for region, project in final_policy.items():
        print(f"  Region {region}: {project}")

--- Problem 2: Project Selection (Maximize Total Profit) ---
Initial Budget: $10M
Maximum Total Profit: $73M
Remaining Budget: $0M
Optimal Investment Plan:
  Region 1: Project 1
  Region 2: Project 2
  Region 3: Project 2
  Region 4: Project 1
  Region 5: Project 3


In [2]:
# Corrected Python Code for Problem 3: Investment Policy
# This code correctly filters the output to only show policies for reachable states.

# Data for Problem 3 (in thousands of dollars)
initial_cash = 10
investment_amount = 10  # Constraint: Only $10k invested each year

investment_A = {
    0: 0.25,  # Return 0 (Lose 10k)
    20: 0.75  # Return 20 (Profit 10k)
}
investment_B = {
    10: 0.9,   # Return 10 (Break even)
    20: 0.1    # Return 20 (Profit 10k)
}

# The maximum possible final cash is 10 (initial) + 3*10 (max profit) = 40k
MAX_STATE = 40
POSSIBLE_STATES = list(range(0, MAX_STATE + 1, 10))
Num_Years = 3

# --- 1. DP for Part 1: Maximize Expected Final Cash ---
# f_k[S] is the max expected total cash from year k to 3, given S at start of year k
f = {Num_Years + 1: {s: s for s in POSSIBLE_STATES}}
policy_expected = {}

for k in range(Num_Years, 0, -1):
    f[k] = {}
    policy_expected[k] = {}

    for S in POSSIBLE_STATES:
        if S < investment_amount:
            # Cannot invest: cash remains S
            f[k][S] = f[k+1].get(S, S)
            policy_expected[k][S] = "No Investment"
            continue

        # Calculate Expected Value for Investment A
        EV_A = 0
        for ret, prob in investment_A.items():
            S_prime = S - investment_amount + ret
            EV_A += prob * f[k+1].get(S_prime, S_prime)

        # Calculate Expected Value for Investment B
        EV_B = 0
        for ret, prob in investment_B.items():
            S_prime = S - investment_amount + ret
            EV_B += prob * f[k+1].get(S_prime, S_prime)

        if EV_A >= EV_B:
            f[k][S] = EV_A
            policy_expected[k][S] = 'A'
        else:
            f[k][S] = EV_B
            policy_expected[k][S] = 'B'

# --- 2. DP for Part 2: Maximize Probability of Final Cash >= 20 ---
# g_k[S] is the max probability of final cash >= 20k from year k to 3
g = {Num_Years + 1: {s: 1.0 if s >= 20 else 0.0 for s in POSSIBLE_STATES}}
policy_prob = {}

for k in range(Num_Years, 0, -1):
    g[k] = {}
    policy_prob[k] = {}

    for S in POSSIBLE_STATES:
        if S < investment_amount:
            g[k][S] = g[k+1].get(S, 0.0)
            policy_prob[k][S] = "No Investment"
            continue

        # Calculate Expected Probability for Investment A
        EP_A = 0
        for ret, prob in investment_A.items():
            S_prime = S - investment_amount + ret
            EP_A += prob * g[k+1].get(S_prime, 0.0)

        # Calculate Expected Probability for Investment B
        EP_B = 0
        for ret, prob in investment_B.items():
            S_prime = S - investment_amount + ret
            EP_B += prob * g[k+1].get(S_prime, 0.0)

        if EP_A >= EP_B:
            g[k][S] = EP_A
            policy_prob[k][S] = 'A'
        else:
            g[k][S] = EP_B
            policy_prob[k][S] = 'B'

# --- Output Results with Filtering ---

# Function to get the set of reachable states for year k
def get_reachable_states(start_state, num_years):
    reachable = {1: {start_state}}
    for k in range(1, num_years):
        reachable[k+1] = set()
        for S in reachable[k]:
            if S >= investment_amount:
                # If invested, possible next states (S - 10 + Return)
                for ret in investment_A.keys(): # Both A and B have returns 0, 10, 20
                    S_next = S - investment_amount + ret
                    if S_next <= MAX_STATE:
                        reachable[k+1].add(S_next)
            else:
                # If not invested, state remains S
                reachable[k+1].add(S)
    return reachable

reachable_states = get_reachable_states(initial_cash, Num_Years)

print("--- Problem 3: Investment Policy (Filtered Output for Initial State $10k) ---")

print("\n--- Part 1: Maximize Expected Final Cash ---")
print(f"Maximum Expected Final Amount (Starting with $10k): ${f[1][initial_cash]:.2f} thousand")
print("Optimal Investment Policy (Year: Total Cash -> Decision):")
for year in range(1, Num_Years + 1):
    print(f"  Year {year}:")
    # Only iterate over states actually reachable in this year
    for S in sorted(reachable_states[year]):
        print(f"    If total cash is ${S}k, invest {policy_expected[year][S]}")

print("\n--- Part 2: Maximize Probability of Final Cash >= $20k ---")
print(f"Maximum Probability (Starting with $10k): {g[1][initial_cash]:.4f}")
print("Optimal Investment Policy (Year: Total Cash -> Decision):")
for year in range(1, Num_Years + 1):
    print(f"  Year {year}:")
    # Only iterate over states actually reachable in this year
    for S in sorted(reachable_states[year]):
        print(f"    If total cash is ${S}k, invest {policy_prob[year][S]}")


--- Problem 3: Investment Policy (Filtered Output for Initial State $10k) ---

--- Part 1: Maximize Expected Final Cash ---
Maximum Expected Final Amount (Starting with $10k): $22.50 thousand
Optimal Investment Policy (Year: Total Cash -> Decision):
  Year 1:
    If total cash is $10k, invest A
  Year 2:
    If total cash is $0k, invest No Investment
    If total cash is $20k, invest A
  Year 3:
    If total cash is $0k, invest No Investment
    If total cash is $10k, invest A
    If total cash is $30k, invest A

--- Part 2: Maximize Probability of Final Cash >= $20k ---
Maximum Probability (Starting with $10k): 0.7975
Optimal Investment Policy (Year: Total Cash -> Decision):
  Year 1:
    If total cash is $10k, invest B
  Year 2:
    If total cash is $0k, invest No Investment
    If total cash is $20k, invest B
  Year 3:
    If total cash is $0k, invest No Investment
    If total cash is $10k, invest A
    If total cash is $30k, invest A
